# Feature generation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import networkx as nx
nx.test()

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

Load parameter settings

In [ ]:
from lynks.helpers import load_config

In [ ]:
experiment_config_dir = Path("../configs")

feat_gen_config_path = experiment_config_dir.joinpath("experiments.toml")

In [ ]:
configuration = load_config(feat_gen_config_path)

In [ ]:
configuration

In [ ]:
from ogb.linkproppred import LinkPropPredDataset

In [ ]:
lp_dataset = LinkPropPredDataset(configuration['dataset_name'])

split_edge = lp_dataset.get_edge_split()

train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]

In [ ]:
from lynks.pipeline import create_pipeline

from lynks.data import create_formatter
from lynks.data import create_graph_builder
from lynks.data import _build_graph
from lynks.sampling import create_sampler

In [ ]:
# preprocessing_config = configuration["preprocessing"]

# data_preprocessing_pipeline = create_pipeline([
#     create_formatter(),
#     create_graph_builder(),
#     create_sampler(
#         method=preprocessing_config["sampling_fn"],
#         n_samples=preprocessing_config['n_samples']
#     ),
# ])


In [ ]:
train_samples = list(map(tuple, train_edge["edge"]))
train_data = (train_edge, train_samples)

In [ ]:
# train_edge["weight"][:1000]

In [ ]:
train_samples = list(map(tuple, train_edge["edge"]))
train_weights = train_edge["weight"]
train_years = train_edge["year"]


# train_samples

In [ ]:

data_processed = data_preprocessing_pipeline(train_data)



In [ ]:
# data_processed.graph.edges

In [ ]:
len(data_processed.samples)

In [ ]:
data_processed.edges[:20]

In [ ]:
# data_processed.samples

In [ ]:
data_processed.labels

In [ ]:
# a = data_processed.nodes[:5]
# b = data_processed.nodes[5:10]

# np.concatenate((a, b))

In [ ]:
from lynks.features import create_graph_topology_featuriser
from lynks.features import create_feature_transform

In [ ]:
feature_config = configuration["features"]

topo_feature_pipeline = create_graph_topology_featuriser(
    common_neighbors_count=feature_config["common_neighbors_count"],
    common_neighbor_centrality=feature_config["common_neighbor_centrality"],
    jaccard_coefficient=feature_config["jaccard_coefficient"],
    adamic_adar_index=feature_config["adamic_adar_index"],
    preferential_attachment=feature_config["preferential_attachment"],
    resource_allocation_index=feature_config["resource_allocation_index"],
    verbose=1
)

In [ ]:
graph, edges, features, feat_labels = topo_feature_pipeline(_build_graph(
    train_samples, train_weights, train_years), train_samples)

In [ ]:
feat_labels

In [ ]:
feats_np = np.concatenate(features, axis=1)


In [ ]:
g = _build_graph(
    train_samples, train_weights, train_years)

# g.nodes

In [ ]:
feature_pipeline = create_pipeline([
    raw_feature_pipeline,
    create_feature_transform(scaling=feature_config["scaling"])
])

In [ ]:
from lynks.logging import get_logger

log = get_logger('test')

In [ ]:
log.info('test')

In [ ]:
# %%time
data_feats = feature_pipeline(data_processed)

In [ ]:
data_feats.feature_names

In [ ]:
data_feats.features

In [ ]:
print(data_feats.features.shape, len(data_feats.labels))

In [ ]:
from lynks.model import create_split_X_Y
from lynks.model import create_train_RandomForestClassifier

In [ ]:

train_model_pipeline = create_pipeline([
    create_split_X_Y(),
    create_train_RandomForestClassifier(101)
])

In [ ]:
rf = train_model_pipeline(data_feats)

In [ ]:
rf

Validation data

In [ ]:
prep_inference_data = create_pipeline([
    create_formatter(),
    create_graph_builder()
])

In [ ]:
validation_samples = list(map(tuple, valid_edge["edge"][:200,:]))
validation_data = (train_edge, validation_samples)

In [ ]:
validation_ds = prep_inference_data(validation_data)

Featurise

In [ ]:
validation_ds_feat = feature_pipeline(validation_ds)

In [ ]:
# infer_pipeline = create_pipeline([
#     rf
# ])

In [ ]:
validation_ds.feature_names

In [ ]:
predictions = rf(validation_ds.features[:, [0,2,3]])

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.metrics import roc_auc_score

In [ ]:
predictions

In [ ]:
confusion_matrix(validation_ds.labels, predictions)

In [ ]:
print(classification_report(validation_ds.labels, predictions))

## Multiprocessing

In [ ]:
# %%time
# data_processed = data_preprocessing_pipeline(train_edge)

In [ ]:
# from lynks.data import split_dataset
# from lynks.data import merge_datasets

In [ ]:
# parallel_datasets = split_dataset(data_processed, n_slices=8)

In [ ]:
# from multiprocessing.pool import ThreadPool as Pool

In [ ]:
# %%time
# with Pool(8) as p:
#     featurized_datasets = p.map(feature_pipeline, parallel_datasets)
    
# dataset_featurised = merge_datasets(featurized_datasets)